# Libraries

In [5]:
import nanohub.remote as nr #Library required to access the remoteDB
import json
import pandas as pd
import os
import math
import numpy as np

# Querying Sim2Ls: meltHEAS

<font color='black' size=4 >Authentication Credentials </font> <font color='red' size=3 > * Do not modify </font>

In [9]:
auth_data = { 'grant_type' : 'tool' }
with open(os.environ["SESSIONDIR"]+"/resources") as file:
    lines = [line.split(" ", 1) for line in file.readlines()]
    properties = {line[0].strip(): line[1].strip() for line in lines if len(line)==2}
    auth_data["sessiontoken"] = properties["session_token"]
    auth_data["sessionnum"] = properties["sessionid"]
    
session = nr.Tools(auth_data)

### Tool Choice: Inputs and Outputs

In [10]:
tool = 'refoxdb'  # Name of the Sim2L you are accessing

req_json = session.requestPost('dbexplorer/dbexplorer/tool_detail?simtool=true', data={'tool': tool}) #Query
req_json = req_json.json() # Turning the resulting request into a JSON file
parameters = req_json['results'] # Extracting parameters (inputs / outputs)

inputs = list(parameters[0][tool]['input'].keys()) # Get list of inputs
print("Things you can query as inputs: \n", inputs)
print("")
outputs = list(parameters[0][tool]['output'].keys()) # Get list of outputs
print("Things you can query as outputs: \n", outputs)

Things you can query as inputs: 
 ['Names', 'affiliations', 'Chemical_Formula', 'Ref_ID', 'Formula_ID', 'Temp', 'MassGain_Unit', 'Time_Unit', 'p_O2', 'p_O2_Unit', 'p_other_gases', 'Disintegration_Time', 'Recession_Rate', 'Oxide_Thickness', 'Oxide_Time', 'Oxides_Formed', 'Phases_Present', 'Experiment_Method', 'Surface_Preparation', 'Heat_Treatment', 'Fabrication_Method', 'Grain_Size', 'gas_flow_rate', 'shape', 'height', 'thickness', 'width', 'Ref', 'Year_Publish', 'Mass_Gain_vs_Time_data']

Things you can query as outputs: 
 ['time', 'min_BIC_model', 'Time_switch_point', 'Rate_constant_1', 'Rate_constant_2']


### Tool Query: Filters and Requests

In [11]:
search = {
    'tool':tool,
    'filters':json.dumps([
        {'field':'input.Temp','operation':'>=','value':'0'},     # Filters included to query [Everything] in the cached database    
        #{'field':'input.Year_Publish','operation':'>=','value':'1962'},   # EXAMPLE of how to filter with an input
    ]),
    'results':json.dumps(['input.Names', 'input.Year_Publish','output.time', 'output.min_BIC_model', 'output.Time_switch_point',
                          'output.Rate_constant_1', 'output.Rate_constant_1']),    
                        # Selected Parameters (Inputs/Outputs) that will be requested from the query
                        # (you can request anything from the previous cell -- make sure you add "input." or "output." appropriately)
    'limit':3500,    
    'revision':0,
 }

req_json = session.requestPost('dbexplorer/dbexplorer/search?simtool=true', data=search, timeout=20) # QUERY
results = req_json.json() # Turning query into JSON file
#print(results)

dataframe = pd.DataFrame(results['results']).dropna() # Deleting columns with NaNs (you can comment this out)
dataframe = dataframe.reset_index(drop=True)

print(dataframe.shape)
display(dataframe) # Complete Dataset

(1146, 7)


,squid,input.Names,input.Year_Publish,output.time,output.min_BIC_model,output.Time_switch_point,output.Rate_constant_1
0,refoxdb/r36/092654417811518bce4306f5f68daa15f1...,,1957.0,"[[0.099675692101747], [0.22934640782052], [0.3...",quadratic-quadratic,1.706563,45.572613
1,refoxdb/r36/6e4478033d807d59d38a62d9283941fc21...,,1956.0,"[[66.570825159081], [167.60178020205], [386.83...",linear-quadratic,6055.000000,0.004567
2,refoxdb/r36/146044012b19a1ec2d08a5fe50b577320a...,,1956.0,"[[40.429994444907], [96.341075487465], [152.99...",linear-linear,1015.000000,0.034316
3,refoxdb/r36/64f81374a2de1b89471602286050cb9beb...,,1956.0,"[[24.92329109091], [122.25778256367], [274.127...",quadratic-linear,2335.000000,0.355401
4,refoxdb/r36/1883fdc89ff5a3cba9d76a4a0db838cb24...,,1956.0,"[[39.97604506481], [53.858512309028], [117.757...",quadratic-linear,1752.000000,1.278545
...,...,...,...,...,...,...,...
1141,refoxdb/r45/9f0306b2f0a7c1c328d6f2cc905ce46947...,nan,1976.0,"[[0.215592709], [0.524780556], [0.87888956], [...",quartic-log,3.968007,2.206352
1142,refoxdb/r45/976298b93ef60e865e20ca4a3b3bea67f8...,nan,2018.0,"[[1.618988959], [4.143346368], [6.807333444], ...",inv_log-quartic,5.462978,-0.061456
1143,refoxdb/r45/32f21b63fc2c6fe6b8d606e5592333fee9...,nan,1958.0,"[[1.2237537367419], [1.4475592488171], [1.6968...",quartic-quadratic,652.408518,1.230147
1144,refoxdb/r45/2db882b4d7be278c0a56d242c7393cf7a0...,nan,1958.0,"[[1.223753737], [1.447559249], [1.69680282], [...",quartic-quadratic,652.408518,1.230147
